# **Derivatives and Option Pricing 40.242**
# **Group Assignment 2**
# **Code Repository**
## Group 4 
## Team Members:
### Nicholas Tan Yi Da (1004126) 
### Praveen Rajesh (1004322) 
### Dody Senputra (1004335) 
### Noel Tan Heng Kiat (1004136) 
### Edric Tjandra (1004306) 

## Import all relevant libraries

In [61]:
# import all the relevant libraries
import numpy as np
from numpy.ma.core import mean
import math
import itertools
# from decimal import Decimal
# from decimal import *

## Initial Parameters

### stock price at t=0, S0 : $300

### annual dividend, q : 4% continuous compunding

### risk free rate, rf : 5% continuous compounding rate

### strike price, X : $310

### days to expiry/simulation period: 480

### annual volatility: 15%


## Monte Carlo Simulation

In [62]:
# common function for monte carlo simulation
np.random.seed(123) # set common random seed for consistency
def monte_carlo(n,option):
    # n: number of iterations
    # option: 0 - put option, 1 - call option
    
    mu, sigma = ( (0.05-0.04-0.5*(0.15**2))*(480/360) , 0.15*((480/360)**0.5) ) # creating mean and stdev parameters for random normal distribution
    payout_list = np.array([])
    simulated_return = np.random.normal(mu, sigma, size=(n))

    if option == 1:
        for i in simulated_return:
            T_price = 300.0*math.exp(i)
            payout_list = np.append(payout_list, max([T_price - 310,0])) # generate list of simulated payouts
    else:
        for i in simulated_return:
            T_price = 300.0*math.exp(i)
            payout_list = np.append(payout_list, max([310 - T_price,0])) # generate list of simulated payouts

    option_price = mean(payout_list) * math.exp(-0.05*(480/360)) # calculate option price

    return option_price

### Number of iterations: 1000, CALL OPTION

In [63]:
np.random.seed(123)
monte_carlo(1000,1)

16.201489034400556

### Number of iterations: 10000, CALL OPTION

In [64]:
np.random.seed(123)
monte_carlo(10000,1)

17.365751955658272

### Number of iterations: 10000, CALL OPTION

In [65]:
np.random.seed(123)
monte_carlo(100000,1)

17.1943873090581

### Number of iterations: 1000, PUT OPTION

In [66]:
np.random.seed(123)
monte_carlo(1000,0)

23.73250928994615

### Number of iterations: 10000, PUT OPTION

In [67]:
np.random.seed(123)
monte_carlo(10000,0)

22.490216306890325

### Number of iterations: 100000, PUT OPTION

In [68]:
np.random.seed(123)
monte_carlo(100000,0)

22.720429457356804

## Binomial Option Pricing

In [71]:
def binomial_tree(n,option):
    # n : number of simulated time steps
    # option: 0 - put option, 1 - call option
    # with reference to John Hull's Option Textbook Chapter 13.8

    delta_t = ((16/12)/n) # discount rate
    prob_pattern = ['u','d'] # u - up, d - down
    prob_pattern_list = [i for i in itertools.combinations_with_replacement(prob_pattern,n)] # generates list of price movement pattern strings
    up_change = math.exp(0.15*math.sqrt(delta_t))
    down_change = math.exp(-0.15*math.sqrt(delta_t))
    up_prob = (math.exp((0.05-0.04)*delta_t) - down_change) / (up_change - down_change) # up movement probability
    down_prob = 1-up_prob # down movement probability
    print("Up/down prob: ", up_prob, down_prob)
    prob_list = [] # list of overall probabilities at final timestep
    price_change_list = [] # list of overall changes to initial stock price at final timestep
    payout_list = None # provides final payouts list at final timestep
    expected_end_payout = 0 # overall expected payout at final timestep

    # print(prob_pattern_list)
    # print(prob_pattern_list[0].count("u")==n)

    for i in prob_pattern_list:
        end_prob = 1
        end_change = 1
        for e in i:
            if e == 'u':
                end_prob = end_prob * up_prob
                end_change = end_change * up_change
            else:
                end_prob = end_prob * down_prob
                end_change = end_change * down_change
        prob_list.append(end_prob)
        price_change_list.append(end_change)

    # print("Prob sum: ",sum(prob_list))

    if option == 1:
        payout_list = [max((300*i)-310,0) for i in price_change_list]
        # print([300*i for i in price_change_list])
        # print(payout_list)
    else:
        payout_list = [max(310-(300*i),0) for i in price_change_list]
        # print([300*i for i in price_change_list])
        # print(payout_list)

    # expected_end_payout = 0

    for i in range(0, len(prob_list)):
        if (prob_pattern_list[i].count('u')== n) or (prob_pattern_list[i].count('d')== n): 
            # capture extreme two ends of binomial option tree and there is always only 1 way to go there. 
            # print(prob_list[i])
            # print(payout_list[i])
            expected_end_payout += prob_list[i]*payout_list[i]
            # print(expected_end_payout)
        else:
            # If the endpoint is not at the middle, there will be n ways to reach the endpoint
            # where n is equal to the timestamp.  
            # print(prob_list[i])
            j = prob_pattern_list[i].count('u')
            # print(j)
            # expected_end_payout += math.comb(n,j)*(prob_list[i])*(payout_list[i])
            expected_end_payout += (math.factorial(n)//math.factorial(j)//math.factorial(n-j))*prob_list[i]*payout_list[i]
            
    #print(expected_end_payout)


    option_price = expected_end_payout* math.exp(-(0.05-0.04)*delta_t) # calculate option price

    return option_price

### Number of timesteps: 100, CALL OPTION

In [72]:
binomial_tree(100,1)

Up/down prob:  0.49951904718094375 0.5004809528190562


18.29925218409521

### Number of timesteps: 500, CALL OPTION

In [73]:
binomial_tree(500,1)

Up/down prob:  0.4997848496787866 0.5002151503212133


18.33169441950317

### Number of timesteps: 1000, CALL OPTION

In [74]:
binomial_tree(1000,1)

Up/down prob:  0.4998478602970041 0.5001521397029959


18.328999387998902

### Number of timesteps: 100, PUT OPTION

In [75]:
binomial_tree(100,0)

Up/down prob:  0.49951904718094375 0.5004809528190562


24.27167022729631

### Number of timesteps: 500, CALL OPTION

In [76]:
binomial_tree(500,0)

Up/down prob:  0.4997848496787866 0.5002151503212133


24.30474955461716

### Number of timesteps: 1000, CALL OPTION

In [77]:
binomial_tree(1000,0)

Up/down prob:  0.4998478602970041 0.5001521397029959


24.302134164392548